# Residential Real Estate Project 1: Simple Linear Regression

In this notebook we will use data on house sales we downloaded from project 0 to predict house prices using simple (one input) linear regression. You will:
* Use python functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself to finish Project 1. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

In [56]:
import numpy as np
import pandas as pd

# Load house sales data

Dataset is from house sales in Santa Clara County, Palo Alto. 

In [57]:
# use pd.read_csv to read data
df=pd.read_csv("santa_clara_county.csv")

check how many houses' sales data you have in the dataset. 

In [58]:
# your code
len(df.index)

350

check what columns are there in this data set.

In [59]:
# your code
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

# Split data into training and testing

There are many ways to split data into training/validation/test sets. One way is to use random split and another common way is to use train_test_split in sklearn. To start with, you can do a 80/20 split. 

In [60]:
# your code
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [61]:
X = df[['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE']]

In [62]:
y = df['PRICE']

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Check how many hosues are in the training dataset and how many houses are in the test dataset.

In [85]:
# your code
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(280, 26) (280,)
(70, 26) (70,)


# Useful python summary functions

In order to make use of the closed form solution as well as take advantage of python's built in functions we will review some important ones. In particular:
* computing the sum of one column in dataframe.
* computing the arithmetic average (mean) of one column in dataframe.
* multiplying column by constants
* multiplying column by other column

In [95]:
# Let's compute the mean of the House Prices in Palo Alto in 2 different ways.
# extract the price column of the sales dataframe
# your code
df['PRICE'] = df['PRICE'].fillna(0)
# first method to calcualte the average price: recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
# notice there are houses with price nan. you need to figure out a way to exclude those houses.
# your code
Total = df['PRICE'].sum()
count_rows = df.shape[0]
average_1 = Total/count_rows

# second method to calcuate the average price, the .mean() function
# your code
average_2 = df['PRICE'].mean()

# print results of both methods, they should be exactly the same
# your code
print('average price via method 1')
print (average_1)
print('average price via method 2')
print (average_2)

average price via method 1
753466.9428571429
average price via method 2
753466.9428571429


As you see, we get the same answer both ways.

In [87]:
# if we want to multiply every price by 0.5 and output it to variable half_prices
# your code
half_prices=df['PRICE']*0.5
# Let's compute the sum of squares of price. First multiply two columns of the same length elementwise with *
# your code
price_squared=df['PRICE']*df['PRICE']
# price_squared is a variable of the squares and we want to add them up and save as sum_prices_squared.
# your code
sum_prices_squared=price_squared.sum()
# print the result
# your code
print('the sum of price squared is')
print (sum_prices_squared)

the sum of price squared is
748070754857044.0


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function

Armed with these python functions we can use the closed form solution to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.
Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [93]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    # your code
    sum_input=input_feature.sum()
    sum_output=output.sum()
    
    # compute the product of the output and the input_feature and its sum
    # your code
    out_input=(input_feature*output).sum()
    
    # compute the squared value of the input_feature and its sum
    # your code
    square_input=(input_feature*input_feature).sum()
    
    # derive the formula for the slope
    # your code
    N = input_feature.shape[0]
    slope = (out_input-sum_input*sum_output/N)/(square_input - input_feature.sum()/N)
    intercept = sum_output/N - slope*sum_input/N
    # derive the formula for the intercept
    # your code
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1*input_feature then we know both our slope and intercept should be 1

In [94]:
test_feature = np.array(range(5))
test_output = 1 + 1*test_feature
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print ("Intercept" )
print (str(test_intercept))
print ("Slope" )
print (str(test_slope))

TypeError: 'int' object is not callable

Now that we know it works let's build a regression model for predicting price based on square feet. Rembember that we train on train_data!

In [92]:
#your code
sqft_intercept,sqft_slope=simple_linear_regression(X_train['SQUARE FEET'],y_train)
print ("Intercept: " + str(sqft_intercept))
print ("Slope: " + str(sqft_slope))

TypeError: 'int' object is not callable

# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using np.array it's easy to multiply an np array by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [80]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    # your code
    predicted_values = intercept + slope * input_feature
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2000 squarefeet according to the squarefeet model we estiamted above

In [81]:
# your code
my_house_sqft = 2000
estimated_price = get_regression_predictions(my_house_sqft,sqft_intercept,sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2000 squarefeet is $782833.51


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output.

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [82]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    # your code
    predictions = input_feature*slope+intercept
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    # your code
    residuals = predictions - output
    # square the residuals and add them up
    # your code
    RSS = (residuals*residuals).sum()
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [83]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

0


According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?

In [84]:
# your code
rss_prices_on_sqft = get_residual_sum_of_squares(X_train['SQUARE FEET'],y_train,sqft_intercept,sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 3.06217346827e+14


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [85]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    # your code
    estimated_feature =(output-intercept)/float(slope)
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

In [86]:
# your code
my_hous_price = 800000
estimated_squarefeed = inverse_regression_predictions(my_house_price,sqft_intercept,sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $1500000.00 is 4145


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales dataframe. Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [87]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bed_intercept,bed_slope=simple_linear_regression(X_train['BEDROOMS'],y_train)
# your code
print "Intercept: " + str(bed_intercept)
print "Slope: " + str(bed_slope)

Intercept: 198499.482394
Slope: 180795.257709


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

Which model(square feet or bedrooms) has lowest RSS on TEST data? 

Think about why this maybe the case.

How about other features in the dataset?

In [88]:
# Compute RSS when using bedrooms on TEST data:
# your code
rss_prices_on_bed = get_residual_sum_of_squares(X_train['BEDROOMS'],y_train,bed_intercept,bed_slope)
print rss_prices_on_bed

8.23783407647e+13


In [89]:
# Compute RSS when using squarefeet on TEST data:
# your code
rss_prices_on_sqft = get_residual_sum_of_squares(X_train['SQUARE FEET'],y_train,sqft_intercept,sqft_slope)
print rss_prices_on_sqft

7.58228652768e+13
